**Hyperparameter Tuning of CNN Architectures with Genetic Algorithm**

**Import Libraries**

In [ ]:
import os
from keras.preprocessing.image import  ImageDataGenerator
from keras.layers import Dense,Flatten,GlobalAveragePooling2D,Input,Conv2D,MaxPooling2D,Dropout
from keras.models import Model,load_model,Sequential
from keras.callbacks import EarlyStopping

**Load Train and Test Dataset**

In [ ]:
#generate training and test images
TARGET_SIZE=(224,224)
INPUT_SIZE=(224,224,3)
BATCHSIZE=32	#could try 128 or 32

#Normalization
train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/Transfer_IDS_IIoT/Datasets/train_A',
        target_size=TARGET_SIZE,
        batch_size=BATCHSIZE,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/Transfer_IDS_IIoT/Datasets/test_A',
        target_size=TARGET_SIZE,
        batch_size=BATCHSIZE,
        class_mode='categorical')

Found 5369 images belonging to 15 classes.
Found 1342 images belonging to 15 classes.


**Define Input Shape and Number of Classes**

In [ ]:
input_shape=INPUT_SIZE
num_class=15


**Tuning of Generic CNN Model**

In [ ]:
#Hyperparameters
# f1,f2,f3 : Filter Sizes
# k : kernel_size
# a1, a2: activation functions
# d1: drop out
# op: optimizer
# ep: number of epochs

def CNN_model( f1, f2, f3, k, a1, a2, d1, op, ep):
  model = Sequential()
  model.add(Conv2D(filters = f1, kernel_size = (k, k), strides=(1,1), input_shape=input_shape,padding='same',activation=a1))
  model.add(Conv2D(filters = f1, kernel_size = (k, k), strides=(1,1), padding='same',activation=a1))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Conv2D(filters = f2, kernel_size = (k, k), strides=(1,1),padding='same',activation = a2))
  model.add(Conv2D(filters = f2, kernel_size = (k, k), strides=(1,1),padding='same',activation = a2))
  model.add(GlobalAveragePooling2D())
  model.add(Dense(units = f3, activation = a2))
  model.add(Dropout(rate=d1))
  model.add(Dense(num_class,activation='softmax'))
  model.compile(loss='categorical_crossentropy',optimizer=op,metrics=['accuracy'])
  es = EarlyStopping(monitor="val_accuracy", patience = 5)
  model.fit(train_generator,steps_per_epoch=len(train_generator),epochs=ep,validation_data=validation_generator,
            validation_steps=len(validation_generator), callbacks=[es], verbose=0)
  return model

In [ ]:
from random import choice
from random import uniform
from numpy.random import randint

In [ ]:
def initialization_cnn():  
  parameters = {}
  f1 = choice([32, 64])
  parameters["f1"] = f1
  f2 = choice([64, 128])
  parameters["f2"] = f2
  f3 = choice([128, 256])
  parameters["f3"] = f3
  k = choice([3,5])
  parameters["k"] = k
  a1 = choice(["relu", "selu", "elu"])
  parameters["a1"] = a1
  a2 = choice(["relu", "selu", "elu"])
  parameters["a2"] = a2
  d1 = round(uniform(0.1, 0.6), 1)
  parameters["d1"] = d1
  op = choice(["adam", "nadam", "adamax", "adagrad"])
  parameters["op"] = op
  ep = randint(10, 25)
  parameters["ep"] = ep
  return parameters

In [ ]:
def generate_population_cnn(n):
  population = []
  for i in range(n):
    chromosome = initialization_cnn()
    population.append(chromosome)
  return population

In [ ]:
# Fitness evaluation metric: Classification Accuracy 
def fitness_evaluation(model):
  metrics = model.evaluate(validation_generator)
  return metrics[1]

In [ ]:
# Roulette wheel selection method
def selection(population_fitness):
  total = sum(population_fitness)
  percentage = [round((x/total) * 100) for x in population_fitness]
  selection_wheel = []
  for pop_index,num in enumerate(percentage):
    selection_wheel.extend([pop_index]*num)
  parent1_ind = choice(selection_wheel)
  parent2_ind = choice(selection_wheel)
  return [parent1_ind, parent2_ind]

In [ ]:
def crossover_cnn(parent1, parent2):
  child1 = {}
  child2 = {}

  child1["f1"] = choice([parent1["f1"], parent2["f1"]])
  child1["f2"] = choice([parent1["f2"], parent2["f2"]])
  child1["f3"] = choice([parent1["f3"], parent2["f3"]])

  child2["f1"] = choice([parent1["f1"], parent2["f1"]])
  child2["f2"] = choice([parent1["f2"], parent2["f2"]])
  child2["f3"] = choice([parent1["f3"], parent2["f3"]])

  child1["k"] = choice([parent1["k"], parent2["k"]])
  child2["k"] = choice([parent1["k"], parent2["k"]])

  child1["a1"] = parent1["a2"]
  child2["a1"] = parent2["a2"]

  child1["a2"] = parent2["a1"]
  child2["a2"] = parent1["a1"]

  child1["d1"] = parent1["d1"]
  child2["d1"] = parent2["d1"]

  child1["op"] = parent2["op"]
  child2["op"] = parent1["op"]

  child1["ep"] = parent1["ep"]
  child2["ep"] = parent2["ep"]
  return [child1, child2]

In [ ]:
def mutation_cnn(chromosome):
  flag = randint(0,40)
  if flag <= 20:
    chromosome["ep"] += randint(0, 10)
  return chromosome

In [ ]:
generations = 5
threshold = 90
num_pop = 5

population = generate_population_cnn(num_pop)

for generation in range(generations):

  population_fitness = []
  for chromosome in population:
    f1 = chromosome["f1"]
    f2 = chromosome["f2"]
    f3 = chromosome["f3"]
    k = chromosome["k"]
    a1 = chromosome["a1"]
    a2 = chromosome["a2"]
    d1 = chromosome["d1"]
    op = chromosome["op"]
    ep = chromosome["ep"]

    try:
      model = CNN_model(f1, f2, f3, k, a1, a2, d1, op, ep)
      acc = fitness_evaluation(model)
      print("Parameters: ", chromosome)
      print("Accuracy: ", round(acc,5))
    except:
      acc=0
      print("Parameters: ", chromosome)
      print("Invalid parameters - Build fail")

    population_fitness.append(acc)
    
  parents_ind = selection(population_fitness)
  parent1 = population[parents_ind[0]]
  parent2 = population[parents_ind[1]]

  children = crossover_cnn(parent1, parent2)
  child1 = mutation_cnn(children[0])
  child2 = mutation_cnn(children[1])

  population.append(child1)
  population.append(child2)

  print("Generation ", generation+1," Outcome: ")
  if max(population_fitness) >= threshold:
    print("Obtained desired accuracy: ", max(population_fitness))
    break
  else:
    print("Maximum accuracy in generation {} : {}".format(generation+1, max(population_fitness)))

  first_min = min(population_fitness)
  first_min_ind = population_fitness.index(first_min)
  population.remove(population[first_min_ind])
  second_min = min(population_fitness)
  second_min_ind = population_fitness.index(second_min)
  population.remove(population[second_min_ind])

42/42 [==============================] - 9s 213ms/step - loss: 1.3277 - accuracy: 0.7004
Parameters:  {'f1': 32, 'f2': 128, 'f3': 128, 'k': 5, 'a1': 'selu', 'a2': 'relu', 'd1': 0.3, 'op': 'nadam', 'ep': 15}
Accuracy:  0.70045
42/42 [==============================] - 11s 254ms/step - loss: 0.0285 - accuracy: 0.9933
Parameters:  {'f1': 32, 'f2': 128, 'f3': 256, 'k': 5, 'a1': 'selu', 'a2': 'selu', 'd1': 0.4, 'op': 'adam', 'ep': 19}
Accuracy:  0.99329
42/42 [==============================] - 7s 157ms/step - loss: 0.0297 - accuracy: 0.9888
Parameters:  {'f1': 64, 'f2': 128, 'f3': 256, 'k': 5, 'a1': 'relu', 'a2': 'relu', 'd1': 0.1, 'op': 'nadam', 'ep': 17}
Accuracy:  0.98882
42/42 [==============================] - 7s 160ms/step - loss: 1.2667 - accuracy: 0.7004
Parameters:  {'f1': 64, 'f2': 64, 'f3': 128, 'k': 3, 'a1': 'elu', 'a2': 'elu', 'd1': 0.3, 'op': 'adagrad', 'ep': 11}
Accuracy:  0.70045
42/42 [==============================] - 8s 197ms/step - loss: 0.1755 - accuracy: 0.9396
Paramete

**Tuning of Xception Model**

In [ ]:
from keras.applications.xception import  Xception

In [ ]:
#Hyperparameters
# frozen: Frozen layers
# f1: Number of units in dense layer
# a1: Activation functions
# d1: Drop out
# op: Optimizer
# ep: Number of epochs

#Define Xception Model
def Xception_model(frozen, f1, a1, d1, op, ep):
  model_fine_tune = Xception(include_top=False, weights='imagenet', input_shape=input_shape)
  for layer in model_fine_tune.layers[:frozen]:           #125
      layer.trainable = False
  for layer in model_fine_tune.layers[frozen:]:
      layer.trainable = True
  model = GlobalAveragePooling2D()(model_fine_tune.output)
  model=Dense(units=f1,activation=a1)(model)        #256
  model=Dropout(d1)(model)                             #0.4
  model = Dense(num_class, activation='softmax')(model)
  model = Model(model_fine_tune.input, model, name='xception')
  model.compile(loss='categorical_crossentropy',optimizer=op,metrics=['accuracy'])
  es = EarlyStopping(monitor="val_accuracy", patience = 2)
  model.fit(train_generator,steps_per_epoch=len(train_generator),epochs=ep,validation_data=validation_generator,
            validation_steps=len(validation_generator), callbacks=[es], verbose=0)
  return model

In [ ]:
from random import choice
from random import uniform
from numpy.random import randint

In [ ]:
def initialization_xception():  
  parameters = {}
  frozen = choice(range(100, 150, 1))
  parameters["frozen"] = frozen
  f1 = choice([128, 256])
  parameters["f1"] = f1
  a1 = choice(["relu", "selu", "elu"])
  parameters["a1"] = a1
  d1 = round(uniform(0.1, 0.6), 1)
  parameters["d1"] = d1
  op = choice(["adam", "nadam", "adamax", "adagrad"])
  parameters["op"] = op
  ep = randint(10, 25)
  parameters["ep"] = ep
  return parameters

In [ ]:
def generate_population_xception(n):
  population = []
  for i in range(n):
    chromosome = initialization_xception()
    population.append(chromosome)
  return population

In [ ]:
# Fitness evaluation metric: Classification Accuracy 
def fitness_evaluation(model):
  metrics = model.evaluate(validation_generator)
  return metrics[1]

In [ ]:
# Roulette wheel selection method
def selection(population_fitness):
  total = sum(population_fitness)
  percentage = [round((x/total) * 100) for x in population_fitness]
  selection_wheel = []
  for pop_index,num in enumerate(percentage):
    selection_wheel.extend([pop_index]*num)
  parent1_ind = choice(selection_wheel)
  parent2_ind = choice(selection_wheel)
  return [parent1_ind, parent2_ind]

In [ ]:
def crossover_xception(parent1, parent2):
  child1 = {}
  child2 = {}

  child1["frozen"] = choice([parent1["frozen"], parent2["frozen"]])
  child2["frozen"] = choice([parent1["frozen"], parent2["frozen"]])

  child1["f1"] = choice([parent1["f1"], parent2["f1"]])
  child2["f1"] = choice([parent1["f1"], parent2["f1"]])

  child1["a1"] = parent2["a1"]
  child2["a1"] = parent1["a1"]

  child1["d1"] = parent1["d1"]
  child2["d1"] = parent2["d1"]

  child1["op"] = parent1["op"]
  child2["op"] = parent2["op"]

  child1["ep"] = parent1["ep"]
  child2["ep"] = parent2["ep"]
  return [child1, child2]

In [ ]:
def mutation_xception(chromosome):
  flag = randint(0,40)
  if flag <= 20:
    chromosome["ep"] += randint(0, 10)
  return chromosome

In [ ]:
generations = 5
threshold = 90
num_pop = 3

population = generate_population_xception(num_pop)

for generation in range(generations):

  population_fitness = []
  for chromosome in population:
    frozen = chromosome["frozen"]
    f1 = chromosome["f1"]
    a1 = chromosome["a1"]
    d1 = chromosome["d1"]
    op = chromosome["op"]
    ep = chromosome["ep"]

    try:
      model = Xception_model(frozen, f1, a1, d1, op, ep)
      acc = fitness_evaluation(model)
      print("Parameters: ", chromosome)
      print("Accuracy: ", round(acc,5))
    except:
      acc=0
      print("Parameters: ", chromosome)
      print("Invalid parameters - Build fail")

    population_fitness.append(acc)
    
  parents_ind = selection(population_fitness)
  parent1 = population[parents_ind[0]]
  parent2 = population[parents_ind[1]]

  children = crossover_xception(parent1, parent2)
  child1 = mutation_xception(children[0])
  child2 = mutation_xception(children[1])

  population.append(child1)
  population.append(child2)

  print("Generation ", generation+1," Outcome: ")
  if max(population_fitness) >= threshold:
    print("Obtained desired accuracy: ", max(population_fitness))
    break
  else:
    print("Maximum accuracy in generation {} : {}".format(generation+1, max(population_fitness)))

  first_min = min(population_fitness)
  first_min_ind = population_fitness.index(first_min)
  population.remove(population[first_min_ind])
  second_min = min(population_fitness)
  second_min_ind = population_fitness.index(second_min)
  population.remove(population[second_min_ind])

42/42 [==============================] - 6s 134ms/step - loss: 0.0304 - accuracy: 0.9888
Parameters:  {'frozen': 149, 'f1': 256, 'a1': 'selu', 'd1': 0.2, 'op': 'adamax', 'ep': 16}
Accuracy:  0.98882
42/42 [==============================] - 6s 135ms/step - loss: 0.0522 - accuracy: 0.9806
Parameters:  {'frozen': 143, 'f1': 256, 'a1': 'elu', 'd1': 0.3, 'op': 'adam', 'ep': 16}
Accuracy:  0.98063
42/42 [==============================] - 6s 134ms/step - loss: 0.0226 - accuracy: 0.9903
Parameters:  {'frozen': 128, 'f1': 128, 'a1': 'selu', 'd1': 0.2, 'op': 'adamax', 'ep': 14}
Accuracy:  0.99031
Generation  1  Outcome: 
Maximum accuracy in generation 1 : 0.9903129935264587
42/42 [==============================] - 6s 135ms/step - loss: 0.0338 - accuracy: 0.9888
Parameters:  {'frozen': 149, 'f1': 256, 'a1': 'selu', 'd1': 0.2, 'op': 'adamax', 'ep': 16}
Accuracy:  0.98882
42/42 [==============================] - 6s 135ms/step - loss: 0.0631 - accuracy: 0.9784
Parameters:  {'frozen': 149, 'f1': 256,

**Tuning of VGG16 Model**

In [ ]:
from keras.applications.vgg16 import VGG16

In [ ]:
#Hyperparameters
# frozen: Frozen layers
# f1: Number of units in dense layer
# a1: Activation functions
# d1: Drop out
# op: Optimizer
# ep: Number of epochs

#Define VGG16 Model
def VGG16_model(frozen, f1, a1, d1, op, ep):
  model_fine_tune = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
  for layer in model_fine_tune.layers[:frozen]:          
      layer.trainable = False
  for layer in model_fine_tune.layers[frozen:]:
      layer.trainable = True
  model = GlobalAveragePooling2D()(model_fine_tune.output)
  model=Dense(units=f1,activation=a1)(model)       
  model=Dropout(d1)(model)                             
  model = Dense(num_class, activation='softmax')(model)
  model = Model(model_fine_tune.input, model, name='vgg')
  model.compile(loss='categorical_crossentropy',optimizer=op,metrics=['accuracy'])
  es = EarlyStopping(monitor="val_accuracy", patience = 4)
  model.fit(train_generator,steps_per_epoch=len(train_generator),epochs=ep,validation_data=validation_generator,
            validation_steps=len(validation_generator), callbacks=[es], verbose=0)
  return model

In [ ]:
from random import choice
from random import uniform
from numpy.random import randint

In [ ]:
def initialization_vgg16():  
  parameters = {}
  #frozen = choice([8, 16, 24])
  frozen = choice(range(8, 25, 1))
  parameters["frozen"] = frozen
  f1 = choice([128, 256])
  parameters["f1"] = f1
  a1 = choice(["relu", "selu", "elu"])
  parameters["a1"] = a1
  d1 = round(uniform(0.1, 0.6), 1)
  parameters["d1"] = d1
  op = choice(["adam", "nadam", "adamax", "adagrad"])
  parameters["op"] = op
  ep = randint(10, 25)
  parameters["ep"] = ep
  return parameters

In [ ]:
def generate_population_vgg16(n):
  population = []
  for i in range(n):
    chromosome = initialization_vgg16()
    population.append(chromosome)
  return population

In [ ]:
# Fitness evaluation metric: Classification Accuracy 
def fitness_evaluation(model):
  metrics = model.evaluate(validation_generator)
  return metrics[1]

In [ ]:
# Roulette wheel selection method
def selection(population_fitness):
  total = sum(population_fitness)
  percentage = [round((x/total) * 100) for x in population_fitness]
  selection_wheel = []
  for pop_index,num in enumerate(percentage):
    selection_wheel.extend([pop_index]*num)
  parent1_ind = choice(selection_wheel)
  parent2_ind = choice(selection_wheel)
  return [parent1_ind, parent2_ind]

In [ ]:
def crossover_vgg16(parent1, parent2):
  child1 = {}
  child2 = {}

  child1["frozen"] = choice([parent1["frozen"], parent2["frozen"]])
  child2["frozen"] = choice([parent1["frozen"], parent2["frozen"]])

  child1["f1"] = choice([parent1["f1"], parent2["f1"]])
  child2["f1"] = choice([parent1["f1"], parent2["f1"]])

  child1["a1"] = parent2["a1"]
  child2["a1"] = parent1["a1"]

  child1["d1"] = parent1["d1"]
  child2["d1"] = parent2["d1"]

  child1["op"] = parent1["op"]
  child2["op"] = parent2["op"]

  child1["ep"] = parent1["ep"]
  child2["ep"] = parent2["ep"]
  return [child1, child2]

In [ ]:
def mutation_vgg16(chromosome):
  flag = randint(0,40)
  if flag <= 20:
    chromosome["ep"] += randint(0, 10)
  return chromosome

In [ ]:
generations = 5
threshold = 90
num_pop = 5

population = generate_population_vgg16(num_pop)

for generation in range(generations):

  population_fitness = []
  for chromosome in population:
    frozen = chromosome["frozen"]
    f1 = chromosome["f1"]
    a1 = chromosome["a1"]
    d1 = chromosome["d1"]
    op = chromosome["op"]
    ep = chromosome["ep"]

    try:
      model = VGG16_model(frozen, f1, a1, d1, op, ep)
      acc = fitness_evaluation(model)
      print("Parameters: ", chromosome)
      print("Accuracy: ", round(acc,5))
    except:
      acc=0
      print("Parameters: ", chromosome)
      print("Invalid parameters - Build fail")

    population_fitness.append(acc)
    
  parents_ind = selection(population_fitness)
  parent1 = population[parents_ind[0]]
  parent2 = population[parents_ind[1]]

  children = crossover_vgg16(parent1, parent2)
  child1 = mutation_vgg16(children[0])
  child2 = mutation_vgg16(children[1])

  population.append(child1)
  population.append(child2)

  print("Generation ", generation+1," Outcome: ")
  if max(population_fitness) >= threshold:
    print("Obtained desired accuracy: ", max(population_fitness))
    break
  else:
    print("Maximum accuracy in generation {} : {}".format(generation+1, max(population_fitness)))

  first_min = min(population_fitness)
  first_min_ind = population_fitness.index(first_min)
  population.remove(population[first_min_ind])
  second_min = min(population_fitness)
  second_min_ind = population_fitness.index(second_min)
  population.remove(population[second_min_ind])

42/42 [==============================] - 7s 161ms/step - loss: 1.0452 - accuracy: 0.7004
Parameters:  {'frozen': 23, 'f1': 128, 'a1': 'elu', 'd1': 0.2, 'op': 'adagrad', 'ep': 18}
Accuracy:  0.70045
42/42 [==============================] - 7s 163ms/step - loss: 0.0135 - accuracy: 0.9948
Parameters:  {'frozen': 8, 'f1': 128, 'a1': 'elu', 'd1': 0.5, 'op': 'adagrad', 'ep': 11}
Accuracy:  0.99478
42/42 [==============================] - 7s 160ms/step - loss: 0.7706 - accuracy: 0.7683
Parameters:  {'frozen': 12, 'f1': 128, 'a1': 'relu', 'd1': 0.5, 'op': 'nadam', 'ep': 24}
Accuracy:  0.76826
42/42 [==============================] - 7s 162ms/step - loss: 0.0221 - accuracy: 0.9903
Parameters:  {'frozen': 16, 'f1': 128, 'a1': 'relu', 'd1': 0.4, 'op': 'adamax', 'ep': 24}
Accuracy:  0.99031
42/42 [==============================] - 7s 161ms/step - loss: 0.0305 - accuracy: 0.9896
Parameters:  {'frozen': 17, 'f1': 256, 'a1': 'elu', 'd1': 0.5, 'op': 'adamax', 'ep': 20}
Accuracy:  0.98957
Generation  1

**Tuning of VGG19 Model**

In [ ]:
from keras.applications.vgg19 import VGG19

In [ ]:
#Hyperparameters
# frozen: Frozen layers
# f1: Number of units in dense layer
# a1: Activation functions
# d1: Drop out
# op: Optimizer
# ep: Number of epochs

#Define VGG19 Model
def VGG19_model(frozen, f1, a1, d1, op, ep):
  model_fine_tune = VGG19(include_top=False, weights='imagenet', input_shape=input_shape)
  for layer in model_fine_tune.layers[:frozen]:          
      layer.trainable = False
  for layer in model_fine_tune.layers[frozen:]:
      layer.trainable = True
  model = GlobalAveragePooling2D()(model_fine_tune.output)
  model=Dense(units=f1,activation=a1)(model)       
  model=Dropout(d1)(model)                             
  model = Dense(num_class, activation='softmax')(model)
  model = Model(model_fine_tune.input, model, name='vgg')
  model.compile(loss='categorical_crossentropy',optimizer=op,metrics=['accuracy'])
  es = EarlyStopping(monitor="val_accuracy", patience = 4)
  model.fit(train_generator,steps_per_epoch=len(train_generator),epochs=ep,validation_data=validation_generator,
            validation_steps=len(validation_generator), callbacks=[es], verbose=0)
  return model

In [ ]:
from random import choice
from random import uniform
from numpy.random import randint

In [ ]:
def initialization_vgg19():  
  parameters = {}
  frozen = choice(range(10, 25, 1))
  parameters["frozen"] = frozen
  f1 = choice([128, 256])
  parameters["f1"] = f1
  a1 = choice(["relu", "selu", "elu"])
  parameters["a1"] = a1
  d1 = round(uniform(0.1, 0.6), 1)
  parameters["d1"] = d1
  op = choice(["adam", "nadam", "adamax", "adagrad"])
  parameters["op"] = op
  ep = randint(10, 25)
  parameters["ep"] = ep
  return parameters

In [ ]:
def generate_population_vgg19(n):
  population = []
  for i in range(n):
    chromosome = initialization_vgg19()
    population.append(chromosome)
  return population

In [ ]:
# Fitness evaluation metric: Classification Accuracy 
def fitness_evaluation(model):
  metrics = model.evaluate(validation_generator)
  return metrics[1]

In [ ]:
# Roulette wheel selection method
def selection(population_fitness):
  total = sum(population_fitness)
  percentage = [round((x/total) * 100) for x in population_fitness]
  selection_wheel = []
  for pop_index,num in enumerate(percentage):
    selection_wheel.extend([pop_index]*num)
  parent1_ind = choice(selection_wheel)
  parent2_ind = choice(selection_wheel)
  return [parent1_ind, parent2_ind]

In [ ]:
def crossover_vgg19(parent1, parent2):
  child1 = {}
  child2 = {}

  child1["frozen"] = choice([parent1["frozen"], parent2["frozen"]])
  child2["frozen"] = choice([parent1["frozen"], parent2["frozen"]])

  child1["f1"] = choice([parent1["f1"], parent2["f1"]])
  child2["f1"] = choice([parent1["f1"], parent2["f1"]])

  child1["a1"] = parent2["a1"]
  child2["a1"] = parent1["a1"]

  child1["d1"] = parent1["d1"]
  child2["d1"] = parent2["d1"]

  child1["op"] = parent1["op"]
  child2["op"] = parent2["op"]

  child1["ep"] = parent1["ep"]
  child2["ep"] = parent2["ep"]
  return [child1, child2]

In [ ]:
def mutation_vgg19(chromosome):
  flag = randint(0,40)
  if flag <= 20:
    chromosome["ep"] += randint(0, 10)
  return chromosome

In [ ]:
generations = 5
threshold = 90
num_pop = 3

population = generate_population_vgg19(num_pop)

for generation in range(generations):

  population_fitness = []
  for chromosome in population:
    frozen = chromosome["frozen"]
    f1 = chromosome["f1"]
    a1 = chromosome["a1"]
    d1 = chromosome["d1"]
    op = chromosome["op"]
    ep = chromosome["ep"]

    try:
      model = VGG19_model(frozen, f1, a1, d1, op, ep)
      acc = fitness_evaluation(model)
      print("Parameters: ", chromosome)
      print("Accuracy: ", round(acc,5))
    except:
      acc=0
      print("Parameters: ", chromosome)
      print("Invalid parameters - Build fail")

    population_fitness.append(acc)
    
  parents_ind = selection(population_fitness)
  parent1 = population[parents_ind[0]]
  parent2 = population[parents_ind[1]]

  children = crossover_vgg19(parent1, parent2)
  child1 = mutation_vgg19(children[0])
  child2 = mutation_vgg19(children[1])

  population.append(child1)
  population.append(child2)

  print("Generation ", generation+1," Outcome: ")
  if max(population_fitness) >= threshold:
    print("Obtained desired accuracy: ", max(population_fitness))
    break
  else:
    print("Maximum accuracy in generation {} : {}".format(generation+1, max(population_fitness)))

  first_min = min(population_fitness)
  first_min_ind = population_fitness.index(first_min)
  population.remove(population[first_min_ind])
  second_min = min(population_fitness)
  second_min_ind = population_fitness.index(second_min)
  population.remove(population[second_min_ind])

42/42 [==============================] - 8s 182ms/step - loss: 0.0089 - accuracy: 0.9963
Parameters:  {'frozen': 10, 'f1': 128, 'a1': 'relu', 'd1': 0.3, 'op': 'adamax', 'ep': 16}
Accuracy:  0.99627
42/42 [==============================] - 8s 182ms/step - loss: 0.0089 - accuracy: 0.9955
Parameters:  {'frozen': 10, 'f1': 256, 'a1': 'selu', 'd1': 0.5, 'op': 'adamax', 'ep': 23}
Accuracy:  0.99553
42/42 [==============================] - 8s 183ms/step - loss: 0.1062 - accuracy: 0.9627
Parameters:  {'frozen': 22, 'f1': 128, 'a1': 'elu', 'd1': 0.5, 'op': 'adamax', 'ep': 20}
Accuracy:  0.96274
Generation  1  Outcome: 
Maximum accuracy in generation 1 : 0.9962742328643799


**Tuning of Inception Model**

In [ ]:
from keras.applications.inception_v3 import InceptionV3

In [ ]:
#Hyperparameters
# frozen: Frozen layers
# f1: Number of units in dense layer
# a1: Activation functions
# d1: Drop out
# op: Optimizer
# ep: Number of epochs

#Define Inception Model
def Inception_model(frozen, f1, a1, d1, op, ep):
  model_fine_tune = InceptionV3(include_top=False, weights='imagenet', input_shape=input_shape)
  for layer in model_fine_tune.layers[:frozen]:          
      layer.trainable = False
  for layer in model_fine_tune.layers[frozen:]:
      layer.trainable = True
  model = GlobalAveragePooling2D()(model_fine_tune.output)
  model=Dense(units=f1,activation=a1)(model)       
  model=Dropout(d1)(model)                             
  model = Dense(num_class, activation='softmax')(model)
  model = Model(model_fine_tune.input, model, name='resnet')
  model.compile(loss='categorical_crossentropy',optimizer=op,metrics=['accuracy'])
  es = EarlyStopping(monitor="val_accuracy", patience = 4)
  model.fit(train_generator,steps_per_epoch=len(train_generator),epochs=ep,validation_data=validation_generator,
            validation_steps=len(validation_generator), callbacks=[es], verbose=0)
  return model

In [ ]:
from random import choice
from random import uniform
from numpy.random import randint

In [ ]:
def initialization_inception():  
  parameters = {}
  frozen = choice(range(30, 50, 1))
  parameters["frozen"] = frozen
  f1 = choice([128, 256])
  parameters["f1"] = f1
  a1 = choice(["relu", "selu", "elu"])
  parameters["a1"] = a1
  d1 = round(uniform(0.1, 0.6), 1)
  parameters["d1"] = d1
  op = choice(["adam", "nadam", "adamax", "adagrad"])
  parameters["op"] = op
  ep = randint(10, 25)
  parameters["ep"] = ep
  return parameters

In [ ]:
def generate_population_inception(n):
  population = []
  for i in range(n):
    chromosome = initialization_inception()
    population.append(chromosome)
  return population

In [ ]:
# Fitness evaluation metric: Classification Accuracy 
def fitness_evaluation(model):
  metrics = model.evaluate(validation_generator)
  return metrics[1]

In [ ]:
# Roulette wheel selection method
def selection(population_fitness):
  total = sum(population_fitness)
  percentage = [round((x/total) * 100) for x in population_fitness]
  selection_wheel = []
  for pop_index,num in enumerate(percentage):
    selection_wheel.extend([pop_index]*num)
  parent1_ind = choice(selection_wheel)
  parent2_ind = choice(selection_wheel)
  return [parent1_ind, parent2_ind]

In [ ]:
def crossover_inception(parent1, parent2):
  child1 = {}
  child2 = {}

  child1["frozen"] = choice([parent1["frozen"], parent2["frozen"]])
  child2["frozen"] = choice([parent1["frozen"], parent2["frozen"]])

  child1["f1"] = choice([parent1["f1"], parent2["f1"]])
  child2["f1"] = choice([parent1["f1"], parent2["f1"]])

  child1["a1"] = parent2["a1"]
  child2["a1"] = parent1["a1"]

  child1["d1"] = parent1["d1"]
  child2["d1"] = parent2["d1"]

  child1["op"] = parent1["op"]
  child2["op"] = parent2["op"]

  child1["ep"] = parent1["ep"]
  child2["ep"] = parent2["ep"]
  return [child1, child2]

In [ ]:
def mutation_inception(chromosome):
  flag = randint(0,40)
  if flag <= 20:
    chromosome["ep"] += randint(0, 10)
  return chromosome

In [ ]:
generations = 5
threshold = 90
num_pop = 3

population = generate_population_inception(num_pop)

for generation in range(generations):

  population_fitness = []
  for chromosome in population:
    frozen = chromosome["frozen"]
    f1 = chromosome["f1"]
    a1 = chromosome["a1"]
    d1 = chromosome["d1"]
    op = chromosome["op"]
    ep = chromosome["ep"]

    try:
      model = Inception_model(frozen, f1, a1, d1, op, ep)
      acc = fitness_evaluation(model)
      print("Parameters: ", chromosome)
      print("Accuracy: ", round(acc,5))
    except:
      acc=0
      print("Parameters: ", chromosome)
      print("Invalid parameters - Build fail")

    population_fitness.append(acc)
    
  parents_ind = selection(population_fitness)
  parent1 = population[parents_ind[0]]
  parent2 = population[parents_ind[1]]

  children = crossover_inception(parent1, parent2)
  child1 = mutation_inception(children[0])
  child2 = mutation_inception(children[1])

  population.append(child1)
  population.append(child2)

  print("Generation ", generation+1," Outcome: ")
  if max(population_fitness) >= threshold:
    print("Obtained desired accuracy: ", max(population_fitness))
    break
  else:
    print("Maximum accuracy in generation {} : {}".format(generation+1, max(population_fitness)))

  first_min = min(population_fitness)
  first_min_ind = population_fitness.index(first_min)
  population.remove(population[first_min_ind])
  second_min = min(population_fitness)
  second_min_ind = population_fitness.index(second_min)
  population.remove(population[second_min_ind])

42/42 [==============================] - 4s 100ms/step - loss: 0.0028 - accuracy: 0.9993
Parameters:  {'frozen': 45, 'f1': 128, 'a1': 'relu', 'd1': 0.5, 'op': 'adagrad', 'ep': 19}
Accuracy:  0.99925
42/42 [==============================] - 4s 98ms/step - loss: 0.0122 - accuracy: 0.9940
Parameters:  {'frozen': 35, 'f1': 128, 'a1': 'relu', 'd1': 0.2, 'op': 'adamax', 'ep': 24}
Accuracy:  0.99404
42/42 [==============================] - 4s 99ms/step - loss: 0.0061 - accuracy: 0.9978
Parameters:  {'frozen': 30, 'f1': 256, 'a1': 'relu', 'd1': 0.6, 'op': 'adagrad', 'ep': 17}
Accuracy:  0.99776
Generation  1  Outcome: 
Maximum accuracy in generation 1 : 0.9992548227310181


**Tuning of InceptionResnetV2 Model**

In [ ]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2

In [ ]:
#Hyperparameters
# frozen: Frozen layers
# f1: Number of units in dense layer
# a1: Activation functions
# d1: Drop out
# op: Optimizer
# ep: Number of epochs

#Define InceptionResnet Model
def InceptionRes_model(frozen, f1, a1, d1, op, ep):
  model_fine_tune = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=input_shape)
  for layer in model_fine_tune.layers[:frozen]:          
      layer.trainable = False
  for layer in model_fine_tune.layers[frozen:]:
      layer.trainable = True
  model = GlobalAveragePooling2D()(model_fine_tune.output)
  model=Dense(units=f1,activation=a1)(model)       
  model=Dropout(d1)(model)                             
  model = Dense(num_class, activation='softmax')(model)
  model = Model(model_fine_tune.input, model, name='resnet')
  model.compile(loss='categorical_crossentropy',optimizer=op,metrics=['accuracy'])
  es = EarlyStopping(monitor="val_accuracy", patience = 4)
  model.fit(train_generator,steps_per_epoch=len(train_generator),epochs=ep,validation_data=validation_generator,
            validation_steps=len(validation_generator), callbacks=[es], verbose=0)
  return model

In [ ]:
from random import choice
from random import uniform
from numpy.random import randint

In [ ]:
def initialization_IncepRes():  
  parameters = {}
  frozen = choice(range(400, 600, 1))
  parameters["frozen"] = frozen
  f1 = choice([128, 256])
  parameters["f1"] = f1
  a1 = choice(["relu", "selu", "elu"])
  parameters["a1"] = a1
  d1 = round(uniform(0.1, 0.6), 1)
  parameters["d1"] = d1
  op = choice(["adam", "nadam", "adamax", "adagrad"])
  parameters["op"] = op
  ep = randint(10, 25)
  parameters["ep"] = ep
  return parameters

In [ ]:
def generate_population_IncepRes(n):
  population = []
  for i in range(n):
    chromosome = initialization_IncepRes()
    population.append(chromosome)
  return population

In [ ]:
# Fitness evaluation metric: Classification Accuracy 
def fitness_evaluation(model):
  metrics = model.evaluate(validation_generator)
  return metrics[1]

In [ ]:
# Roulette wheel selection method
def selection(population_fitness):
  total = sum(population_fitness)
  percentage = [round((x/total) * 100) for x in population_fitness]
  selection_wheel = []
  for pop_index,num in enumerate(percentage):
    selection_wheel.extend([pop_index]*num)
  parent1_ind = choice(selection_wheel)
  parent2_ind = choice(selection_wheel)
  return [parent1_ind, parent2_ind]

In [ ]:
def crossover_IncepRes(parent1, parent2):
  child1 = {}
  child2 = {}

  child1["frozen"] = choice([parent1["frozen"], parent2["frozen"]])
  child2["frozen"] = choice([parent1["frozen"], parent2["frozen"]])

  child1["f1"] = choice([parent1["f1"], parent2["f1"]])
  child2["f1"] = choice([parent1["f1"], parent2["f1"]])

  child1["a1"] = parent2["a1"]
  child2["a1"] = parent1["a1"]

  child1["d1"] = parent1["d1"]
  child2["d1"] = parent2["d1"]

  child1["op"] = parent1["op"]
  child2["op"] = parent2["op"]

  child1["ep"] = parent1["ep"]
  child2["ep"] = parent2["ep"]
  return [child1, child2]

In [ ]:
def mutation_IncepRes(chromosome):
  flag = randint(0,40)
  if flag <= 20:
    chromosome["ep"] += randint(0, 10)
  return chromosome

In [ ]:
generations = 5
threshold = 90
num_pop = 3

population = generate_population_IncepRes(num_pop)

for generation in range(generations):

  population_fitness = []
  for chromosome in population:
    frozen = chromosome["frozen"]
    f1 = chromosome["f1"]
    a1 = chromosome["a1"]
    d1 = chromosome["d1"]
    op = chromosome["op"]
    ep = chromosome["ep"]

    try:
      model = InceptionRes_model(frozen, f1, a1, d1, op, ep)
      acc = fitness_evaluation(model)
      print("Parameters: ", chromosome)
      print("Accuracy: ", round(acc,5))
    except:
      acc=0
      print("Parameters: ", chromosome)
      print("Invalid parameters - Build fail")

    population_fitness.append(acc)
    
  parents_ind = selection(population_fitness)
  parent1 = population[parents_ind[0]]
  parent2 = population[parents_ind[1]]

  children = crossover_IncepRes(parent1, parent2)
  child1 = mutation_IncepRes(children[0])
  child2 = mutation_IncepRes(children[1])

  population.append(child1)
  population.append(child2)

  print("Generation ", generation+1," Outcome: ")
  if max(population_fitness) >= threshold:
    print("Obtained desired accuracy: ", max(population_fitness))
    break
  else:
    print("Maximum accuracy in generation {} : {}".format(generation+1, max(population_fitness)))

  first_min = min(population_fitness)
  first_min_ind = population_fitness.index(first_min)
  population.remove(population[first_min_ind])
  second_min = min(population_fitness)
  second_min_ind = population_fitness.index(second_min)
  population.remove(population[second_min_ind])

42/42 [==============================] - 8s 180ms/step - loss: 0.0052 - accuracy: 0.9978
Parameters:  {'frozen': 451, 'f1': 128, 'a1': 'selu', 'd1': 0.3, 'op': 'adamax', 'ep': 12}
Accuracy:  0.99776
42/42 [==============================] - 8s 177ms/step - loss: 0.0112 - accuracy: 0.9955
Parameters:  {'frozen': 408, 'f1': 256, 'a1': 'selu', 'd1': 0.2, 'op': 'adamax', 'ep': 12}
Accuracy:  0.99553
42/42 [==============================] - 8s 178ms/step - loss: 0.2455 - accuracy: 0.9620
Parameters:  {'frozen': 567, 'f1': 256, 'a1': 'relu', 'd1': 0.4, 'op': 'adam', 'ep': 24}
Accuracy:  0.962
Generation  1  Outcome: 
Maximum accuracy in generation 1 : 0.997764527797699


**Tuning of EfficientNetB7 Model**

In [ ]:
from keras.applications.efficientnet import EfficientNetB7

In [ ]:
#Hyperparameters
# frozen: Frozen layers
# f1: Number of units in dense layer
# a1: Activation functions
# d1: Drop out
# op: Optimizer
# ep: Number of epochs

#Define EfficientNetB7 Model
def EfficientNetB7_model(frozen, f1, a1, d1, op, ep):
  model_fine_tune = EfficientNetB7(include_top=False, weights='imagenet', input_shape=input_shape)
  for layer in model_fine_tune.layers[:frozen]:          
      layer.trainable = False
  for layer in model_fine_tune.layers[frozen:]:
      layer.trainable = True
  model = GlobalAveragePooling2D()(model_fine_tune.output)
  model=Dense(units=f1,activation=a1)(model)       
  model=Dropout(d1)(model)                             
  model = Dense(num_class, activation='softmax')(model)
  model = Model(model_fine_tune.input, model, name='efficientnetb7')
  model.compile(loss='categorical_crossentropy',optimizer=op,metrics=['accuracy'])
  es = EarlyStopping(monitor="val_accuracy", patience = 4)
  model.fit(train_generator,steps_per_epoch=len(train_generator),epochs=ep,validation_data=validation_generator,
            validation_steps=len(validation_generator), callbacks=[es], verbose=0)
  return model

In [ ]:
from random import choice
from random import uniform
from numpy.random import randint

In [ ]:
def initialization_EfficientNetB7():  
  parameters = {}
  frozen = choice(range(270, 350, 1))
  parameters["frozen"] = frozen
  f1 = choice([128, 256])
  parameters["f1"] = f1
  a1 = choice(["relu", "selu", "elu"])
  parameters["a1"] = a1
  d1 = round(uniform(0.1, 0.6), 1)
  parameters["d1"] = d1
  op = choice(["adam", "nadam", "adamax", "adagrad"])
  parameters["op"] = op
  ep = randint(10, 25)
  parameters["ep"] = ep
  return parameters

In [ ]:
def generate_population_EfficientNetB7(n):
  population = []
  for i in range(n):
    chromosome = initialization_EfficientNetB7()
    population.append(chromosome)
  return population

In [ ]:
# Fitness evaluation metric: Classification Accuracy 
def fitness_evaluation(model):
  metrics = model.evaluate(validation_generator)
  return metrics[1]

In [ ]:
# Roulette wheel selection method
def selection(population_fitness):
  total = sum(population_fitness)
  percentage = [round((x/total) * 100) for x in population_fitness]
  selection_wheel = []
  for pop_index,num in enumerate(percentage):
    selection_wheel.extend([pop_index]*num)
  parent1_ind = choice(selection_wheel)
  parent2_ind = choice(selection_wheel)
  return [parent1_ind, parent2_ind]

In [ ]:
def crossover_EfficientNetB7(parent1, parent2):
  child1 = {}
  child2 = {}

  child1["frozen"] = choice([parent1["frozen"], parent2["frozen"]])
  child2["frozen"] = choice([parent1["frozen"], parent2["frozen"]])

  child1["f1"] = choice([parent1["f1"], parent2["f1"]])
  child2["f1"] = choice([parent1["f1"], parent2["f1"]])

  child1["a1"] = parent2["a1"]
  child2["a1"] = parent1["a1"]

  child1["d1"] = parent1["d1"]
  child2["d1"] = parent2["d1"]

  child1["op"] = parent1["op"]
  child2["op"] = parent2["op"]

  child1["ep"] = parent1["ep"]
  child2["ep"] = parent2["ep"]
  return [child1, child2]

In [ ]:
def mutation_EfficientNetB7(chromosome):
  flag = randint(0,40)
  if flag <= 20:
    chromosome["ep"] += randint(0, 10)
  return chromosome

In [ ]:
generations = 5
threshold = 90
num_pop = 3

population = generate_population_EfficientNetB7(num_pop)

for generation in range(generations):

  population_fitness = []
  for chromosome in population:
    frozen = chromosome["frozen"]
    f1 = chromosome["f1"]
    a1 = chromosome["a1"]
    d1 = chromosome["d1"]
    op = chromosome["op"]
    ep = chromosome["ep"]

    try:
      model = EfficientNetB7_model(frozen, f1, a1, d1, op, ep)
      acc = fitness_evaluation(model)
      print("Parameters: ", chromosome)
      print("Accuracy: ", round(acc,5))
    except:
      acc=0
      print("Parameters: ", chromosome)
      print("Invalid parameters - Build fail")

    population_fitness.append(acc)
    
  parents_ind = selection(population_fitness)
  parent1 = population[parents_ind[0]]
  parent2 = population[parents_ind[1]]

  children = crossover_EfficientNetB7(parent1, parent2)
  child1 = mutation_EfficientNetB7(children[0])
  child2 = mutation_EfficientNetB7(children[1])

  population.append(child1)
  population.append(child2)

  print("Generation ", generation+1," Outcome: ")
  if max(population_fitness) >= threshold:
    print("Obtained desired accuracy: ", max(population_fitness))
    break
  else:
    print("Maximum accuracy in generation {} : {}".format(generation+1, max(population_fitness)))

  first_min = min(population_fitness)
  first_min_ind = population_fitness.index(first_min)
  population.remove(population[first_min_ind])
  second_min = min(population_fitness)
  second_min_ind = population_fitness.index(second_min)
  population.remove(population[second_min_ind])

42/42 [==============================] - 15s 355ms/step - loss: 0.0643 - accuracy: 0.9709
Parameters:  {'frozen': 302, 'f1': 256, 'a1': 'elu', 'd1': 0.2, 'op': 'nadam', 'ep': 18}
Accuracy:  0.97094
42/42 [==============================] - 15s 355ms/step - loss: 0.1417 - accuracy: 0.9687
Parameters:  {'frozen': 285, 'f1': 128, 'a1': 'selu', 'd1': 0.2, 'op': 'adamax', 'ep': 18}
Accuracy:  0.9687
42/42 [==============================] - 15s 355ms/step - loss: 0.0952 - accuracy: 0.9694
Parameters:  {'frozen': 288, 'f1': 128, 'a1': 'selu', 'd1': 0.4, 'op': 'adam', 'ep': 15}
Accuracy:  0.96945
Generation  1  Outcome: 
Maximum accuracy in generation 1 : 0.9709389209747314
42/42 [==============================] - 15s 355ms/step - loss: 13.8226 - accuracy: 0.0380
Parameters:  {'frozen': 302, 'f1': 256, 'a1': 'elu', 'd1': 0.2, 'op': 'nadam', 'ep': 18}
Accuracy:  0.038
42/42 [==============================] - 15s 355ms/step - loss: 0.0882 - accuracy: 0.9583
Parameters:  {'frozen': 288, 'f1': 128,

**Tuning of EfficientNetV2L Model**

In [ ]:
from keras.applications.efficientnet_v2 import EfficientNetV2L

In [ ]:
#Hyperparameters
# frozen: Frozen layers
# f1: Number of units in dense layer
# a1: Activation functions
# d1: Drop out
# op: Optimizer
# ep: Number of epochs

#Define EfficientNetV2L Model
def EfficientNetV2L_model(frozen, f1, a1, d1, op, ep):
  model_fine_tune = EfficientNetV2L(include_top=False, weights='imagenet', input_shape=input_shape)
  for layer in model_fine_tune.layers[:frozen]:          
      layer.trainable = False
  for layer in model_fine_tune.layers[frozen:]:
      layer.trainable = True
  model = GlobalAveragePooling2D()(model_fine_tune.output)
  model=Dense(units=f1,activation=a1)(model)       
  model=Dropout(d1)(model)                             
  model = Dense(num_class, activation='softmax')(model)
  model = Model(model_fine_tune.input, model, name='efficientnetv2-l')
  model.compile(loss='categorical_crossentropy',optimizer=op,metrics=['accuracy'])
  es = EarlyStopping(monitor="val_accuracy", patience = 4)
  model.fit(train_generator,steps_per_epoch=len(train_generator),epochs=ep,validation_data=validation_generator,
            validation_steps=len(validation_generator), callbacks=[es], verbose=0)
  return model

In [ ]:
from random import choice
from random import uniform
from numpy.random import randint

In [ ]:
def initialization_EfficientNetV2L():  
  parameters = {}
  frozen = choice(range(380, 430, 1))
  parameters["frozen"] = frozen
  f1 = choice([128, 256])
  parameters["f1"] = f1
  a1 = choice(["relu", "selu", "elu"])
  parameters["a1"] = a1
  d1 = round(uniform(0.1, 0.6), 1)
  parameters["d1"] = d1
  op = choice(["adam", "nadam", "adamax", "adagrad"])
  parameters["op"] = op
  ep = randint(10, 25)
  parameters["ep"] = ep
  return parameters

In [ ]:
def generate_population_EfficientNetV2L(n):
  population = []
  for i in range(n):
    chromosome = initialization_EfficientNetV2L()
    population.append(chromosome)
  return population

In [ ]:
# Fitness evaluation metric: Classification Accuracy 
def fitness_evaluation(model):
  metrics = model.evaluate(validation_generator)
  return metrics[1]

In [ ]:
# Roulette wheel selection method
def selection(population_fitness):
  total = sum(population_fitness)
  percentage = [round((x/total) * 100) for x in population_fitness]
  selection_wheel = []
  for pop_index,num in enumerate(percentage):
    selection_wheel.extend([pop_index]*num)
  parent1_ind = choice(selection_wheel)
  parent2_ind = choice(selection_wheel)
  return [parent1_ind, parent2_ind]

In [ ]:
def crossover_EfficientNetV2L(parent1, parent2):
  child1 = {}
  child2 = {}

  child1["frozen"] = choice([parent1["frozen"], parent2["frozen"]])
  child2["frozen"] = choice([parent1["frozen"], parent2["frozen"]])

  child1["f1"] = choice([parent1["f1"], parent2["f1"]])
  child2["f1"] = choice([parent1["f1"], parent2["f1"]])

  child1["a1"] = parent2["a1"]
  child2["a1"] = parent1["a1"]

  child1["d1"] = parent1["d1"]
  child2["d1"] = parent2["d1"]

  child1["op"] = parent1["op"]
  child2["op"] = parent2["op"]

  child1["ep"] = parent1["ep"]
  child2["ep"] = parent2["ep"]
  return [child1, child2]

In [ ]:
def mutation_EfficientNetV2L(chromosome):
  flag = randint(0,40)
  if flag <= 20:
    chromosome["ep"] += randint(0, 10)
  return chromosome

In [ ]:

generations = 5
threshold = 90
num_pop = 3

population = generate_population_EfficientNetB7(num_pop)

for generation in range(generations):

  population_fitness = []
  for chromosome in population:
    frozen = chromosome["frozen"]
    f1 = chromosome["f1"]
    a1 = chromosome["a1"]
    d1 = chromosome["d1"]
    op = chromosome["op"]
    ep = chromosome["ep"]

    try:
      model = EfficientNetV2L_model(frozen, f1, a1, d1, op, ep)
      acc = fitness_evaluation(model)
      print("Parameters: ", chromosome)
      print("Accuracy: ", round(acc,5))
    except:
      acc=0
      print("Parameters: ", chromosome)
      print("Invalid parameters - Build fail")

    population_fitness.append(acc)
    
  parents_ind = selection(population_fitness)
  parent1 = population[parents_ind[0]]
  parent2 = population[parents_ind[1]]

  children = crossover_EfficientNetV2L(parent1, parent2)
  child1 = mutation_EfficientNetV2L(children[0])
  child2 = mutation_EfficientNetV2L(children[1])

  population.append(child1)
  population.append(child2)

  print("Generation ", generation+1," Outcome: ")
  if max(population_fitness) >= threshold:
    print("Obtained desired accuracy: ", max(population_fitness))
    break
  else:
    print("Maximum accuracy in generation {} : {}".format(generation+1, max(population_fitness)))

  first_min = min(population_fitness)
  first_min_ind = population_fitness.index(first_min)
  population.remove(population[first_min_ind])
  second_min = min(population_fitness)
  second_min_ind = population_fitness.index(second_min)
  population.remove(population[second_min_ind])

42/42 [==============================] - 16s 384ms/step - loss: 0.0305 - accuracy: 0.9851
Parameters:  {'frozen': 305, 'f1': 256, 'a1': 'selu', 'd1': 0.5, 'op': 'adam', 'ep': 24}
Accuracy:  0.9851
42/42 [==============================] - 16s 386ms/step - loss: 0.0234 - accuracy: 0.9911
Parameters:  {'frozen': 284, 'f1': 256, 'a1': 'selu', 'd1': 0.3, 'op': 'adamax', 'ep': 15}
Accuracy:  0.99106
42/42 [==============================] - 16s 386ms/step - loss: 0.0212 - accuracy: 0.9918
Parameters:  {'frozen': 316, 'f1': 128, 'a1': 'selu', 'd1': 0.5, 'op': 'adam', 'ep': 19}
Accuracy:  0.9918
Generation  1  Outcome: 
Maximum accuracy in generation 1 : 0.9918032884597778
42/42 [==============================] - 16s 386ms/step - loss: 0.0648 - accuracy: 0.9829
Parameters:  {'frozen': 316, 'f1': 128, 'a1': 'selu', 'd1': 0.5, 'op': 'adam', 'ep': 19}
Accuracy:  0.98286
